In [1]:
import random
import itertools
from IPython.display import clear_output
import time

In [2]:
ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
suits = ['Spades', 'Clubs', 'Hearts', 'Diamonds']
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
          'Nine':9, 'Ten':10, 'Jack':11, 'Queen':12, 'King':13, 'Ace': 14}

In [3]:
class Card:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.value = values[self.rank]
    
    def __repr__(self):
        return self.rank + ' ' + self.suit
    
    def __str__(self):
        return self.rank + ' ' + self.suit

In [4]:
class Deck:
    
    def __init__(self):
        self.deck = []
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(rank, suit))
                
    def shuffle(self):
        random.shuffle(self.deck)
    
    def burn(self):
        self.deck.append(self.deck.pop(0)) # move first element to end of list
    
    def deal(self):
        return self.deck.pop(0)
    
    def __str__(self):
        deck = ''
        for card in self.deck:
            deck += card.__str__() + '\n'
        return deck[:-1]

In [22]:
class Player:
    
    def __init__(self, name, chips, hole_cards, in_hand, not_acted, curr_bet, all_in):
        self.name = name
        self.chips = chips
        self.hole_cards = hole_cards
        self.in_hand = in_hand
        self.not_acted = not_acted
        self.curr_bet = curr_bet
        self.all_in = all_in
        
    def fold(self):
        self.in_hand = False
        self.curr_bet = 0
    
    def bet(self, amnt):
        self.chips -= amnt
        self.curr_bet += amnt
        
    def chip_transaction(self, amnt):
        self.chips += amnt
    
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name

In [23]:
def best_hand(cards):
    possible_hands = []
    for hand in list(itertools.combinations(cards, 5)):
        possible_hands.append(list(hand))
    possible_hands.sort(key = (lambda x: x[0].value + x[1].value + x[2].value + x[3].value + x[4].value), reverse = True)
    for hand in possible_hands:
        hand.sort(key = (lambda x: x.value), reverse = True) 
    
    # royal flush and straight flush
    for hand in possible_hands:
        if hand[0].suit == hand[1].suit == hand[2].suit == hand[3].suit == hand[4].suit:
            if hand[0].rank == 'Ace' and hand[1].rank == 'King' and hand[2].rank == 'Queen' and hand[3].rank == 'Jack' and hand[4].rank == 'Ten':
                return "royal flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
            elif hand[0] == (hand[1].value + 1) == (hand[2].value + 2) == (hand[3].value + 3) == (hand[4].value + 4):
                return "straight flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
            
    # quads
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank == hand[2].rank == hand[3].rank:
            return "four of a kind", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank == hand[3].rank == hand[4].rank:
            return "four of a kind", [hand[1], hand[2], hand[3], hand[4], hand[0]]
        
    # full house
    for hand in possible_hands:
        if (hand[0].rank == hand[1].rank) and (hand[2].rank == hand[3].rank == hand[4].rank):
            return "full house", [hand[2], hand[3], hand[3], hand[0], hand[1]]
        if (hand[0].rank == hand[1].rank == hand[2].rank) and (hand[3].rank == hand[4].rank):
            return "full house", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        
    # flush
    for hand in possible_hands:
        if hand[0].suit == hand[1].suit == hand[2].suit == hand[3].suit == hand[4].suit:
            return "flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        
    # straight
    for hand in possible_hands:
        if hand[0].rank == 'Ace' and hand[1].rank == 'Five' and hand[2].rank == 'Four' and hand[3].rank == 'Three' and hand[2].rank == 'Two':
            return "straight", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        elif hand[0] == (hand[1].value + 1) == (hand[2].value + 2) == (hand[3].value + 3) == (hand[4].value + 4):
            return "straight", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        
    # trips
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank == hand[2].rank:
            return "three of a kind", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank == hand[3].rank:
            return "three of a kind", [hand[1], hand[2], hand[3], hand[0], hand[4]]
        if hand[2].rank == hand[3].rank == hand[4].rank:
            return "three of a kind", [hand[2], hand[3], hand[4], hand[0], hand[1]]
        
    # two pair
    for hand in possible_hands:
        if (hand[0].rank == hand[1].rank) and (hand[2].rank == hand[3].rank):
            return "two pair", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if (hand[0].rank == hand[1].rank) and (hand[3].rank == hand[4].rank):
            return "two pair", [hand[0], hand[1], hand[3], hand[4], hand[2]]
        if (hand[1].rank == hand[2].rank) and (hand[3].rank == hand[4].rank):
            return "two pair", [hand[1], hand[2], hand[3], hand[4], hand[0]]
        
    # one pair
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank:
            return "one pair", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank:
            return "one pair", [hand[1], hand[2], hand[0], hand[3], hand[4]]
        if hand[2].rank == hand[3].rank:
            return "one pair", [hand[2], hand[3], hand[0], hand[1], hand[4]]
        if hand[3].rank == hand[4].rank:
            return "one pair", [hand[3], hand[4], hand[0], hand[1], hand[2]]
        
    hand = possible_hands[0]
    return 'high card', [hand[0], hand[1], hand[2], hand[3], hand[4]]

In [24]:
def hole_cards(players, deck):
    for i in range(2): # deal 2 hole cards to each player
        for player in players:
            player.hole_cards.append(deck.deal())
    for player in players: # view cards
        view = input(f"Press enter to view {player.name}'s cards.")
        print(player.hole_cards)
        exit = input("Press enter to stop viewing.")
        clear_output()
        time.sleep(0.05)
    return players, deck

In [25]:
def count_players_in_hand(players):
    count = 0
    for player in players:
        if player.in_hand:
            count += 1
    return count

In [26]:
def betting_round(players, starting_position, curr_bet, pot, bb):
    
    i = starting_position # index of first player to act
    
    betting = True
    while betting:
        
        player = players[i]
        
        # player action
        if player.in_hand and player.not_acted:
            invalid_action = True
            while invalid_action:
                
                print(f"{player.name} Action: ")
                move = input("Enter one of 'fold' / 'check' / 'call' / 'raise' / 'all-in': ")
                
                # fold
                if move[0] == 'f':
                    invalid_action = False
                    player.fold()
                    player.not_acted = False
                
                # check and call
                elif move[0] == 'c':    
                    if move[1] == 'h': # check
                        if player.curr_bet == curr_bet:
                            invalid_action = False
                            player.not_acted = False
                        else:
                            print("Invalid Action; You must match current bet or fold.") 
                    else: # call
                        if player.curr_bet < curr_bet and player.chips >= (curr_bet - player.curr_bet):
                            invalid_action = False
                            pot += (curr_bet - player.curr_bet)
                            player.bet(curr_bet - player.curr_bet)
                            player.not_acted = False
                        elif player.curr_bet < curr_bet and player.chips < (curr_bet - player.curr_bet):
                            move = 'all in' # if insufficient chips to call, player shoves
                        else: # if player calls when no one has bet, player checks
                            invalid_action = False
                            player.not_acted = False
                            
                # raise
                elif move[0] == 'r':
                    if curr_bet > 0 and player.chips <= (2*curr_bet):
                        move = 'a'
                    elif curr_bet == 0 and player.chips <= bb:
                        move = 'a'
                    else:
                        invalid_action = False
                        invalid_raise = True
                        while invalid_raise:
                            amnt = int(input('Enter total number of chips to bet(includes current bet): '))
                            if curr_bet > 0:
                                if amnt >= (2*curr_bet):
                                    invalid_raise = False
                                    pot += (amnt - player.curr_bet)
                                    player.bet(amnt - player.curr_bet)
                                    curr_bet = amnt
                                    player.not_acted = False
                                    for player in players:
                                        if player.in_hand and player != players[i] and not(player.all_in):
                                            player.not_acted = True 
                                else:
                                    print('Invalid Raise Amount; You must at least double current bet.')
                            else:
                                if amnt >= bb:
                                    invalid_raise = False
                                    pot += (amnt - player.curr_bet)
                                    player.bet(amnt)
                                    curr_bet = amnt
                                    player.not_acted = False
                                    for player in players:
                                        if player.in_hand and player != players[i] and not(player.all_in):
                                            player.not_acted = True        
                                else:
                                    print('Invalid Raise Amount; Your bet must be greater than or equal to big blind.')
                
                elif move[0] != 'a':
                    print('Invalid Action')
                
                if move[0] == 'a': # all in
                    invalid_action = False
                    if (player.chips + player.curr_bet) > curr_bet:
                        curr_bet = player.chips + player.curr_bet
                        for player in players:
                            if player.in_hand and player != players[i] and not(player.all_in):
                                player.not_acted = True
                    player.bet(player.chips)
                    player.all_in = True
                    pot += player.chips
                    player.not_acted = False
        
        # after each action, check if betting is finished
        betting = False
        if count_players_in_hand(players) > 1:
            for player in players:
                if player.not_acted: # CONTINUE FROM HERE
                    betting = True
                    break
        
        # move to next player
        if players[i] == players[-1]:
            i = 0
        else:
            i += 1
        
    clear_output()
    time.sleep(0.05)
        
    return players, pot

In [27]:
def detect_winner(round_, players, pot): # if everyone has folded, declare last man standing as winner
    if count_players_in_hand(players) == 1:
        round_ = False
        for player in players:
            if player.in_hand:
                print(player.name + " wins " + str(pot) + " chips.")
                player.chip_transaction(pot)
    return round_, players

In [28]:
def community_cards(cards, num, deck, pot):
    deck.burn()
    for i in range(num):
        cards.append(deck.deal())
    print(cards)
    print(f"Pot: {pot}\n")
    return cards, deck

In [29]:
def reset_curr_bets(players):
    for player in players:
        if player.in_hand:
            player.not_acted = True
            player.curr_bet = 0
    return players

In [30]:
def showdown_winner(players, river, pot):
    if count_players_in_hand(players) != 1:
        possible_hands = ['royal flush', 'straight flush', 'four of a kind', 'full house', 'flush', 'straight', 'three of a kind', 'two pair', 'one pair', 'high card']
        hands = {}
        for player in players:
            if player.in_hand:
                cards = player.hole_cards + river
                value_lst = [] # convert list of card to list of values of cards
                for card in best_hand(cards)[1]:
                    value_lst.append(card.value)
                hands[player] = ((best_hand(cards)[0], value_lst, best_hand(cards)[1]), possible_hands.index(best_hand(cards)[0]))
        best = min(hands.items(), key = (lambda x: x[1][1]))[1][1]
        for player in hands.keys():
            if hands[player][1] != best:
                hands[player] = (('0', [0, 0, 0, 0, 0]), 10) # effectively deletes this player
        
        # finding best kicker (sorted by value_lst)
        winner = sorted(hands.items(), key = (lambda x: x[1][0][1]), reverse = True)[0][0]

        # reporting winner
        print(f"{winner.name} hole cards are {winner.hole_cards}.")
        print(f"Community cards are {river}.\n")
        print(f"{winner.name}'s hand: {hands[winner][0][2]}")
        print(winner.name + " wins " + str(pot) + " chips with " + hands[winner][0][0])
        winner.chip_transaction(pot)
        
    return players

In [32]:
def play_again(players, game):
    again = input("\nDo you want to play again?")
    if again.lower()[0] == 'y':
        game = True
        players = players[1:] + [players[0]] # position progression
        for player in players: # reset
            player.hole_cards = []
            player.in_hand = True
            player.not_acted = True
            player.curr_bet = 0
            player.all_in = False
        clear_output()
        time.sleep(0.05)
    else:
        game = False
        clear_output()
        time.sleep(0.1)
        print("Thanks for playing!")
        for player in players:
            print(f"{player.name}: {player.chips}")
    return players, game

In [34]:
sb = int(input("Choose small blind amount (enter a number): "))
bb = sb*2
stack = int(input("Choose starting stack amount (enter a number): "))
num_players = int(input("How many players? (enter a number -> min 3 and max 9): "))

players = []
for player in range(num_players):
    name = input(f"Enter player {player}'s name: ")
    players.append(Player(name, stack, [], True, True, 0, False))
random.shuffle(players)

game = True
while game:
    
    round_ = True
    
    # positions
    print("Order of Action: ")
    for player in players:
        print(f"{player.name}: {player.chips}")
    continue_prompt = input("Press enter to continue: ")
    clear_output()
    time.sleep(0.05)
    
    # pre-flop
    if round_:
        deck = Deck() # new deck
        deck.shuffle() # shuffle deck
        
        players, deck = hole_cards(players, deck) # deal hole cards
        players[0].bet(sb) # small blind posted
        players[1].bet(bb) # big blind posted
        players, pot = betting_round(players, 2, bb, (sb + bb), bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # flop
    if round_:
        flop, deck = community_cards([], 3, deck, pot) # deal flop
        players = reset_curr_bets(players) # reset current bets
        players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # turn
    if round_:
        turn, deck = community_cards(flop, 1, deck, pot) # deal turn
        players = reset_curr_bets(players) # reset current bets
        players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # river
    if round_:
        river, deck = community_cards(turn, 1, deck, pot) # deal river
        players = reset_curr_bets(players) # reset current bets
        players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown
        players = showdown_winner(players, river, pot) # determine best hand at showdown
    
    players, game = play_again(players, game) # another round

KeyboardInterrupt: Interrupted by user